## Evaluations

Ein Dataset mit QA Pairs erstellen.


In [1]:
from langsmith import Client

client = Client()

# Define dataset: these are your test cases
dataset_name = "QA Jungle Dataset"
dataset = client.create_dataset(dataset_name)
client.create_examples(
    inputs=[
        {"question": "Was ist ein Jaguar?"},
        {"question": "Was ist ein Tukan?"},
        {"question": "Was ist ein Faultier?"},
        {"question": "Was ist ein Anakonda?"},
        {"question": "Was ist ein Ozelot?"},
    ],
    outputs=[
        {"answer": "Ein großes Raubtier, das in den tropischen Wäldern Amerikas lebt"},
        {"answer": "Ein bunter Vogel mit einem großen Schnabel, der im Regenwald lebt"},
        {"answer": "Ein langsames Säugetier, das die meiste Zeit an Bäumen hängt"},
        {"answer": "Eine riesige Schlange, die im Amazonas-Regenwald lebt"},
        {
            "answer": "Ein Axolotl ist ein neotenes Amphibium, das seine larvalen Merkmale wie Kiemen auch im erwachsenen Stadium behält und in Mexiko beheimatet ist."
        },
    ],
    dataset_id=dataset.id,
)

Als nächstes erstellen wir einen Evaluator der generierte Antworten mit den Antworten aus dem Dataset vergleicht.


In [2]:
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts.prompt import PromptTemplate
from langsmith.evaluation import LangChainStringEvaluator

_PROMPT_TEMPLATE = """You are an expert professor specialized in grading students' answers to questions.
You are grading the following question:
{query}
Here is the real answer:
{answer}
You are grading the following predicted answer:
{result}
Respond with CORRECT or INCORRECT:
Grade:
"""

PROMPT = PromptTemplate(
    input_variables=["query", "answer", "result"], template=_PROMPT_TEMPLATE
)
eval_llm = ChatAnthropic(model="claude-3-sonnet-20240229", temperature=0.0)

qa_evaluator = LangChainStringEvaluator(
    "qa", config={"llm": eval_llm, "prompt": PROMPT}
)

Jetzt erstellen wir die Funktion, die getestet werden soll. Im Beispiel ein einfacher LLM Aufruf mit einer einzelnen Frage.


In [3]:
from langchain.schema import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI


def my_app(question):
    llm = ChatOpenAI(model="gpt-4o-mini")
    prompt = SystemMessage(
        content="Antworte auf die Frage des Nutzer in einer kurzen, präzisen Art (ein Satz)."
    )

    result = llm.invoke(
        [
            prompt,
            HumanMessage(content=question),
        ]
    )
    return result.content


def langsmith_app(inputs):
    output = my_app(inputs["question"])
    return {"output": output}

Jetzt müssen wir das Experiment nur noch ausführen und schon können wir uns die Ergebnisse in Langsmith ansehen.


In [4]:
from langsmith.evaluation import evaluate

experiment_results = evaluate(
    langsmith_app,
    data=dataset_name,
    evaluators=[qa_evaluator],
    experiment_prefix="jungle_questions",
)

View the evaluation results for experiment: 'jungle_questions-8ffefffe' at:
https://smith.langchain.com/o/eb86f40f-e90c-4eb8-846e-94f09f6daf2e/datasets/820274a3-9cd3-4a56-b956-cf9ee10baa34/compare?selectedSessions=5868391d-9964-4360-9ab6-68d3b3d1a774




0it [00:00, ?it/s]